Scrape sigma Project (if you want you can try to scrape amazon instead):
Sigma:
https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch

Amazon:
https://www.amazon.com/s?k=iphone&crid=2OOD54QR2XQFT&sprefix=ipho%2Caps%2C285&ref=nb_sb_noss_2

In [41]:
import requests
from bs4 import BeautifulSoup
response = requests.get("https://www.sigma-computer.com/search?search=laptop&submit_search=&route=product%2Fsearch")
records:list[tuple]=[]
if response.status_code == 200:
    soup = BeautifulSoup(response.content,"html.parser")
    productTags= soup.find_all("div",attrs={"class":"product-item-container"})
    for e in productTags:
         description = e.find("div",attrs={"class","description"}).find("a").find_all("p")[0].text
         if "Processor" in description :
            title = e.find("div",attrs={"class","caption"}).find("a").attrs["title"]
            stock = e.find("span",attrs={"class","stock"}).text if e.find("span",attrs={"class","stock"}) != None else "NO"
            oldPrice = e.find("span",attrs={"class","price-old"}).text.split()[0]
            newPrice = e.find("span",attrs={"class","price-new"}).text.split()[0]
            description = description.replace("\r"," ").replace("\n"," ").replace(",","-").strip()
            records.append((title,stock,oldPrice,newPrice,description))

    with open("Laptops.csv",mode="w",encoding="utf-8") as df:
        df.write("title,stock,oldPrice,newPrice,description\n")
        for e in records:
            df.write(f"\"{e[0]}\",\"{e[1]}\",{e[2]},{e[3]},\"{e[4]}\"\n")


   
    
   

# Git Hub URL
https://github.com/AWasseem/Epsilon/tree/main/Session%2012%20datacollection